In [ ]:
import matplotlib.pyplot as plt
import sklearn
from sklearn import datasets, manifold

In [ ]:
sr_points, sr_color = datasets.make_swiss_roll(n_samples=1500, noise=0.05)

In [ ]:
fig = plt.figure()

ax = fig.add_subplot(projection='3d')
plt.rcParams.update({'font.size': 20})
ax.scatter(sr_points[:,0],
           sr_points[:,1],
           sr_points[:,2],
           c=sr_color, s=30, alpha = 0.8,cmap='jet')
plt.title("Swiss roll dataset in 3d")
ax.view_init(azim=-70, elev=15)
plt.show()

### The solution of unrolling the doe

In [ ]:
import torch
colors_tensor = torch.from_numpy(sr_color)
#colors_normalized = torch.nn.functional.normalize(colors_tensor,dim=0)
y_tensor = torch.from_numpy(sr_points[:,1])
#y_normalized = torch.nn.functional.normalize(y_tensor,dim=0)

plt.rcParams.update({'font.size': 20})
#plt.figure(figsize=(20,10),dpi=100)
plt.title("The unrolled swiss roll: projection on the\n 2nd and the 4th coordinate (the color)")
plt.xlabel("the y-coordinate of sr_points")
plt.ylabel("sr_color")
#plt.scatter(y_normalized,colors_normalized,c=sr_color)
plt.scatter(y_tensor,colors_tensor,c=sr_color,cmap='jet',s=30)
#
plt.show()

### LLE and t-SNE

In [ ]:
sr_lle, sr_err = manifold.locally_linear_embedding(
    sr_points, n_neighbors=12, n_components=2
)

sr_tsne = manifold.TSNE(n_components=2, perplexity=40, random_state=0).fit_transform(
    sr_points
)
"""
fig, axs = plt.subplots(figsize=(8, 8), nrows=2)
axs[0].scatter(sr_lle[:, 0], sr_lle[:, 1], c=sr_color)
axs[0].set_title("LLE Embedding of Swiss Roll")
axs[1].scatter(sr_tsne[:, 0], sr_tsne[:, 1], c=sr_color)
_ = axs[1].set_title("t-SNE Embedding of Swiss Roll")
"""


In [ ]:
plt.rcParams.update({'font.size': 20})
plt.scatter(sr_lle[:, 0], sr_lle[:, 1], c=sr_color,cmap='jet',s=30)
plt.title("LLE Embedding of the swiss roll")
plt.show()

In [ ]:
plt.rcParams.update({'font.size': 20})
plt.scatter(sr_tsne[:, 0], sr_tsne[:, 1], c=sr_color,cmap='jet',s=30)
plt.title("t-SNE embedding of the swiss roll")
plt.show()

### UMAP

In [ ]:
import umap

In [ ]:
mapper = umap.UMAP().fit(sr_points)

In [ ]:
encoded_points = mapper.embedding_

In [ ]:
plt.rcParams.update({'font.size': 20}) # makes all fonts on the plot be 20
plt.scatter( encoded_points[:,0], encoded_points[:,1], c=sr_color, alpha=0.5, cmap='jet',marker='o',edgecolors=None )
plt.title( "UMAP embedding of the swiss roll")
#plt.colorbar(orientation='vertical',shrink = 0.7)
plt.show()